# Introdução ao CoolProp
## Cursos de Verão IFMG-Arcos 2023
### Rotinas computacionais aplicadas às Ciências Térmicas - GESESC
#### Autor: *Reginaldo Gonçalves Leão Junior*

CoolProp é uma biblioteca escrita em C++ com um *wrapers* para diversas linguagens, inclusive Python, em um ambiente Anaconda compatível ela pode ser instalada utilizando-se:

```
conda install -c conda-forge coolprop
```

Ela disponibiliza propriedades termodinâmicas de diversos compostos de forma gratuíta. Dentre as várias funcionalidades da biblioteca, a classe `PropsSI` permite a consulta de propriedades termodinâmmicas no SI.